In [ ]:
import git
from pathlib import Path
import os

ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

DATASET = "coco"
FINAL_DATA_NAME = 'approx1e5-coco-outdoor-fourier' # + channel
CONSTANT_SAMPLE_SIZE = int(1e5)
RAW_DATA_SUFFIX = "coco-cropped-Outdoor"
SAVE_DF = False

data_dir = os.path.join(ROOT_DIR, 'raw-data','coco')
file_list = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(data_dir)
data_dir
BATCH_NUM = 0

In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from transform import *
os.chdir(os.path.join(ROOT_DIR, "dataset-preparation"))
freq_df = pd.read_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"), index_col= ["dataset", "transform", "group"])


In [4]:
file_list = [os.path.join(data_dir, f"toy-{RAW_DATA_SUFFIX}", filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(os.path.join(data_dir, f"toy-{RAW_DATA_SUFFIX}"))
file_names[:5]

['000000335177.jpg',
 '000000278705.jpg',
 '000000568981.jpg',
 '000000092416.jpg',
 '000000173830.jpg']

In [4]:
'''Assuming No batching is required. Not applicable for agriVision'''

# data_dir = os.path.join(ROOT_DIR, "raw-data", "agriVision", "full-agriVision-RGB-cleaned")

# for channel in ['red', 'blue', 'green', 'gray', 'infrared']:

#     channel_fr = convert_to_fourier_basis(data_dir, channel, debug = True)
#     pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"full-agriVision-fourier-{channel}-df.pickle"))

#     min_group, max_group = 2, max(channel_fr['band'])
#     group_data_map = dict()
#     group_data_map_size = dict()
#     for group in np.arange(min_group, max_group + 1):
#         data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
#         group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
#         group_data_map_size[group] = data.size
    
#     pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
#     pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
    

'Assuming No batching is required. Not applicable for agriVision'

In [5]:
'''To split large dataset into many batches, only needs to be run once'''
# k = 10000
# target_dir = os.path.join(ROOT_DIR, 'raw-data', 'agriVision') # Where the batch{i} folders will be created
# directorySplit(folder_dir = data_dir, target_dir = target_dir, name = RAW_DATA_SUFFIX, k = k)
# print(f"{len(file_names)//k} batches created" )

'To split large dataset into many batches, only needs to be run once'

In [10]:
'''Show all subsets of data in raw data folder that have already been created'''
print(''.join([x+"\n" for x in os.listdir(data_dir) if x.__contains__(RAW_DATA_SUFFIX) and not x.__contains__("._")]))


toy-coco-cropped-Outdoor



In [11]:
def npz_opener_pickle(path):
    image = Image.open(path).convert('RGB')
    return np.array(image)

# Fourier

In [12]:
#Values obtained from plots in coco_indoor_Fourier_NEW.ipynb
STARTING_VALUE = 0.026203921611325662
ENDING_VALUE =0.6138159492317302
MULT_FACTOR = 1.18
if BATCH_NUM is None:
    batch_dir = os.path.join(ROOT_DIR, "raw-data", "coco", f"toy-{RAW_DATA_SUFFIX}")
else:
    batch_dir = os.path.join(ROOT_DIR, "raw-data", "coco", f"batch{BATCH_NUM}-{RAW_DATA_SUFFIX}")
splits = getSplits(STARTING_VALUE,ENDING_VALUE, MULT_FACTOR)
print(splits)
BATCH_NUM = None

[0.026203921611325662, 0.03092062750136428, 0.036486340451609846, 0.04305388173289962, 0.05080358044482155, 0.05994822492488942, 0.07073890541136951, 0.08347190838541602, 0.09849685189479089, 0.11622628523585324, 0.1371470165783068, 0.16183347956240202, 0.19096350588363437, 0.22533693694268853, 0.2658975855923724, 0.3137591509989994, 0.3702357981788193, 0.43687824185100677, 0.5155163253841879, 0.6083092639533417, 0.7178049314649432]


In [13]:
TRANSFORM = "fourier"
channel = "red"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener_pickle)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = np.mean(channel_fr[(channel_fr['band'] == group)]['magnitude_endpoints'].iloc[0])
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))

if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()


100%|██████████| 2446/2446 [00:11<00:00, 212.30it/s]


[0.02620392 0.03092063 0.03648634 0.04305388 0.05080358 0.05994822
 0.07073891 0.08347191 0.09849685 0.11622629 0.13714702 0.16183348
 0.19096351 0.22533694 0.26589759 0.31375915 0.3702358  0.43687824
 0.51551633 0.60830926 0.71780493]


100%|██████████| 21/21 [00:03<00:00,  6.99it/s]


,band,channel,magnitude_endpoints,unique_magnitudes,data
0,1,red,"(0.0, 0.025012204052472065)",22,"[28482.855, 30711.96, 35676.426, 29848.79, 389..."
1,2,red,"(0.026203921611325662, 0.030508787796510367)",7,"[65.72201, -254.89946, -53.10038, 250.94365, -..."
2,3,red,"(0.03125, 0.03601384553630034)",10,"[30.199188, 84.70316, 555.7424, -144.31247, -7..."
3,4,red,"(0.036851488797096106, 0.04296875)",13,"[-66.74939, -64.2344, 70.77693, 32.897762, -10..."
4,5,red,"(0.04314594147338774, 0.05078125)",17,"[273.52667, 453.15985, -8.039323, 134.58537, -..."


In [13]:
del channel_fr, group_data_map, group_data_map_size

In [14]:
TRANSFORM = "fourier"
channel = "blue"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener_pickle)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = np.mean(channel_fr[(channel_fr['band'] == group)]['magnitude_endpoints'].iloc[0])
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))

if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()

100%|██████████| 1604/1604 [00:04<00:00, 371.61it/s]


[0.02620392 0.03223082 0.03964391 0.04876201 0.05997728 0.07377205
 0.09073962 0.11160973 0.13727997 0.16885437 0.20769087 0.25545977
 0.31421552 0.38648509 0.47537666 0.58471329 0.71919734]


100%|██████████| 17/17 [00:01<00:00,  9.05it/s]


,band,channel,magnitude_endpoints,unique_magnitudes,data
0,1,blue,"(0.0, 0.025012204052472065)",22,"[25380.016, 30578.777, 43920.64, 22871.48, 249..."
1,2,blue,"(0.026203921611325662, 0.03221176270013797)",10,"[-29.592005, -5.743617, 269.38425, -99.818245,..."
2,3,blue,"(0.03314563036811941, 0.039257326645003476)",13,"[10.993307, -31.40373, -139.4958, -7.939845, -..."
3,4,blue,"(0.03983608994994363, 0.048317644050206957)",19,"[-120.00683, -54.023926, -365.25037, -63.58884..."
4,5,blue,"(0.04894517221149089, 0.05975413492491545)",28,"[11.270843, -51.959522, 25.22712, -34.639797, ..."


In [15]:
del channel_fr, group_data_map, group_data_map_size

In [16]:
TRANSFORM = "fourier"
channel = "green"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener_pickle)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = np.mean(channel_fr[(channel_fr['band'] == group)]['magnitude_endpoints'].iloc[0])
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))
    
if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()

100%|██████████| 1604/1604 [00:04<00:00, 371.89it/s]


[0.02620392 0.03223082 0.03964391 0.04876201 0.05997728 0.07377205
 0.09073962 0.11160973 0.13727997 0.16885437 0.20769087 0.25545977
 0.31421552 0.38648509 0.47537666 0.58471329 0.71919734]


100%|██████████| 17/17 [00:01<00:00,  8.73it/s]


,band,channel,magnitude_endpoints,unique_magnitudes,data
0,1,green,"(0.0, 0.025012204052472065)",22,"[24739.984, 34184.09, 45530.055, 27241.96, 308..."
1,2,green,"(0.026203921611325662, 0.03221176270013797)",10,"[-91.89867, -166.39987, 243.79117, -100.061554..."
2,3,green,"(0.03314563036811941, 0.039257326645003476)",13,"[-3.8628771, 10.944305, -136.06499, 13.759761,..."
3,4,green,"(0.03983608994994363, 0.048317644050206957)",19,"[-151.74765, -10.343752, -296.91342, -40.27313..."
4,5,green,"(0.04894517221149089, 0.05975413492491545)",28,"[-6.313587, -67.33274, -3.907643, -10.753381, ..."


In [17]:
del channel_fr, group_data_map, group_data_map_size

In [18]:
TRANSFORM = "fourier"
channel = "gray"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener_pickle)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = np.mean(channel_fr[(channel_fr['band'] == group)]['magnitude_endpoints'].iloc[0])
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))
    
if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()

100%|██████████| 1604/1604 [00:05<00:00, 294.53it/s]


[0.02620392 0.03223082 0.03964391 0.04876201 0.05997728 0.07377205
 0.09073962 0.11160973 0.13727997 0.16885437 0.20769087 0.25545977
 0.31421552 0.38648509 0.47537666 0.58471329 0.71919734]


100%|██████████| 17/17 [00:01<00:00,  8.88it/s]


,band,channel,magnitude_endpoints,unique_magnitudes,data
0,1,gray,"(0.0, 0.025012204052472065)",22,"[25300.414, 33779.24, 46227.543, 28610.145, 30..."
1,2,gray,"(0.026203921611325662, 0.03221176270013797)",10,"[-102.580414, -145.0102, 237.34221, -74.68768,..."
2,3,gray,"(0.03314563036811941, 0.039257326645003476)",13,"[-33.4497, -42.67274, -123.34379, 17.246386, 3..."
3,4,gray,"(0.03983608994994363, 0.048317644050206957)",19,"[-157.2597, -57.384617, -313.06015, -34.103252..."
4,5,gray,"(0.04894517221149089, 0.05975413492491545)",28,"[14.109035, -68.36598, -6.147733, -23.62347, 1..."


In [19]:
del channel_fr, group_data_map, group_data_map_size

# Wavelet

In [16]:
FINAL_DATA_NAME = 'approx1e5-agriVision-wavelet'
if BATCH_NUM is None:
    batch_dir = os.path.join(ROOT_DIR, "raw-data", "agriVision", f"full-{RAW_DATA_SUFFIX}")
else:
    batch_dir = os.path.join(ROOT_DIR, "raw-data", "agriVision", f"batch{BATCH_NUM}-{RAW_DATA_SUFFIX}")

In [17]:
TRANSFORM = "wavelet"
channel = "red"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0])
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group)]['frequency'].iloc[0]
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))
                             
if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

10 layers being used


100%|██████████| 4500/4500 [01:33<00:00, 48.35it/s]


,channel,layer,frequency,orientation,data
0,red,1,0.099611,L1,"[12.37854, 388.76587, -71.011154, -525.40607, ..."
1,red,2,0.110679,D,"[-56.85054, 10.112775, -65.61645, 3.6756237, -..."
2,red,2,0.110679,H,"[426.77856, 36.807053, 2.2234993, 41.115826, 3..."
3,red,2,0.110679,V,"[-458.58713, 52.57764, -45.266056, -4.2271876,..."
4,red,3,0.124514,D,"[-192.24008, 1.2320688, -6.463406, 114.53845, ..."


In [18]:
del channel_wv, group_data_map, group_data_map_size

In [19]:
TRANSFORM = "wavelet"
channel = "green"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0])
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group)]['frequency'].iloc[0]
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))

if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

10 layers being used


 31%|███▏      | 1416/4500 [00:32<01:11, 43.29it/s]


KeyboardInterrupt: 

In [ ]:
del channel_wv, group_data_map, group_data_map_size

In [ ]:
TRANSFORM = "wavelet"
channel = "blue"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0])
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group)]['frequency'].iloc[0]
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))
    
if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

8 layers being used


100%|██████████| 1590/1590 [00:04<00:00, 365.56it/s]


,channel,layer,frequency,orientation,data
0,blue,1,0.110679,L1,"[0.20727114, -33.177452, -23.717556, -104.1030..."
1,blue,2,0.124514,D,"[-26.155533, 6.1044655, 21.780111, 24.545807, ..."
2,blue,2,0.124514,H,"[-90.20362, -75.99399, 19.310474, -11.829856, ..."
3,blue,2,0.124514,V,"[76.4653, 12.720116, -29.556787, -10.373663, 0..."
4,blue,3,0.142301,D,"[53.602417, 3.668749, -16.509668, -0.57486, -1..."


In [ ]:
del channel_wv, group_data_map, group_data_map_size

In [ ]:
TRANSFORM = "wavelet"
channel = "gray"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0])
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group)]['frequency'].iloc[0]
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))

if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

8 layers being used


100%|██████████| 1590/1590 [00:04<00:00, 339.12it/s]


,channel,layer,frequency,orientation,data
0,gray,1,0.110679,L1,"[-6.1360106, 35.824085, -12.069928, -92.59161,..."
1,gray,2,0.124514,D,"[-20.23139, 5.5501356, 18.585161, 14.010576, -..."
2,gray,2,0.124514,H,"[-54.903618, -72.86237, 8.412701, -6.4397974, ..."
3,gray,2,0.124514,V,"[55.306328, 9.492829, -24.910969, -6.03024, 0...."
4,gray,3,0.142301,D,"[36.00795, 3.1733897, -9.826171, -1.8638246, -..."


In [ ]:
del channel_wv, group_data_map, group_data_map_size